In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Association Rules Exploration\n",
    "## FP-Growth Algorithm for Basket Optimization\n",
    "\n",
    "This notebook explores association rules discovered from grocery transaction data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('../')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import pickle\n",
    "import json\n",
    "\n",
    "# Set style\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)\n",
    "\n",
    "print(\" Imports successful\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load preprocessed baskets\n",
    "with open('../data/processed_baskets.pkl', 'rb') as f:\n",
    "    data = pickle.load(f)\n",
    "\n",
    "baskets = data['baskets']\n",
    "sequences = data['sequences']\n",
    "stats = data['statistics']\n",
    "\n",
    "print(f\" Dataset Statistics:\")\n",
    "for key, value in stats.items():\n",
    "    print(f\"   {key}: {value}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Load Association Rules"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load association rules\n",
    "with open('../models/association_rules.pkl', 'rb') as f:\n",
    "    model_data = pickle.load(f)\n",
    "\n",
    "rules_df = model_data['rules_df']\n",
    "\n",
    "print(f\" Loaded {len(rules_df)} association rules\")\n",
    "print(f\"\\n Top 10 Rules:\\n\")\n",
    "rules_df.head(10)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Visualize Rule Metrics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Scatter plot: Support vs Confidence\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 5))\n",
    "\n",
    "# Plot 1: Support vs Confidence\n",
    "axes[0].scatter(rules_df['support'], rules_df['confidence'], \n",
    "                alpha=0.5, c=rules_df['lift'], cmap='viridis')\n",
    "axes[0].set_xlabel('Support')\n",
    "axes[0].set_ylabel('Confidence')\n",
    "axes[0].set_title('Association Rules: Support vs Confidence')\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# Plot 2: Lift distribution\n",
    "axes[1].hist(rules_df['lift'], bins=30, color='steelblue', edgecolor='black')\n",
    "axes[1].set_xlabel('Lift')\n",
    "axes[1].set_ylabel('Frequency')\n",
    "axes[1].set_title('Distribution of Lift Values')\n",
    "axes[1].axvline(x=1.0, color='red', linestyle='--', label='Lift = 1.0')\n",
    "axes[1].legend()\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Analyze Top Rules"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Format top rules nicely\n",
    "top_rules = rules_df.head(20).copy()\n",
    "\n",
    "# Convert frozensets to strings\n",
    "top_rules['Rule'] = top_rules.apply(\n",
    "    lambda row: f\"{list(row['antecedents'])[0]} → {list(row['consequents'])[0]}\",\n",
    "    axis=1\n",
    ")\n",
    "\n",
    "# Create visualization\n",
    "fig, ax = plt.subplots(figsize=(12, 8))\n",
    "\n",
    "y_pos = np.arange(len(top_rules))\n",
    "ax.barh(y_pos, top_rules['confidence'], color='steelblue')\n",
    "ax.set_yticks(y_pos)\n",
    "ax.set_yticklabels(top_rules['Rule'])\n",
    "ax.invert_yaxis()\n",
    "ax.set_xlabel('Confidence')\n",
    "ax.set_title('Top 20 Association Rules by Confidence')\n",
    "ax.grid(True, axis='x', alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Test Recommendations"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from ml_engine.association.fp_growth import AssociationEngine\n",
    "\n",
    "# Load trained engine\n",
    "engine = AssociationEngine()\n",
    "engine.load('../models/association_rules.pkl')\n",
    "\n",
    "# Test with sample baskets\n",
    "test_baskets = [\n",
    "    ['Pantry'],\n",
    "    ['Dairy', 'Bakery'],\n",
    "    ['Produce', 'Meat'],\n",
    "    ['Pantry', 'Produce', 'Dairy']\n",
    "]\n",
    "\n",
    "print(\" Testing Recommendations:\\n\")\n",
    "print(\"=\"*60)\n",
    "\n",
    "for basket in test_baskets:\n",
    "    print(f\"\\n Current Basket: {basket}\")\n",
    "    recommendations = engine.get_recommendations(basket, max_items=3)\n",
    "    \n",
    "    if recommendations:\n",
    "        print(\"   Recommendations:\")\n",
    "        for item, metrics in recommendations.items():\n",
    "            print(f\"   • {item}\")\n",
    "            print(f\"     Confidence: {metrics['confidence']:.1%}\")\n",
    "            print(f\"     Support: {metrics['support']:.1%}\")\n",
    "            print(f\"     Lift: {metrics['lift']:.2f}\")\n",
    "    else:\n",
    "        print(\"    No recommendations found\")\n",
    "    print(\"-\"*60)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Network Visualization (Optional)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize top rules as network\n",
    "# Note: Requires networkx (pip install networkx)\n",
    "\n",
    "try:\n",
    "    import networkx as nx\n",
    "    \n",
    "    # Create directed graph\n",
    "    G = nx.DiGraph()\n",
    "    \n",
    "    # Add top 15 rules\n",
    "    for _, row in rules_df.head(15).iterrows():\n",
    "        antecedent = list(row['antecedents'])[0]\n",
    "        consequent = list(row['consequents'])[0]\n",
    "        weight = row['confidence']\n",
    "        \n",
    "        G.add_edge(antecedent, consequent, weight=weight)\n",
    "    \n",
    "    # Draw network\n",
    "    plt.figure(figsize=(14, 10))\n",
    "    pos = nx.spring_layout(G, k=2, iterations=50)\n",
    "    \n",
    "    # Draw nodes\n",
    "    nx.draw_networkx_nodes(G, pos, node_color='lightblue', \n",
    "                          node_size=2000, alpha=0.9)\n",
    "    \n",
    "    # Draw edges with varying width based on confidence\n",
    "    edges = G.edges()\n",
    "    weights = [G[u][v]['weight'] * 5 for u, v in edges]\n",
    "    nx.draw_networkx_edges(G, pos, width=weights, alpha=0.6, \n",
    "                          edge_color='gray', arrows=True, \n",
    "                          arrowsize=20, arrowstyle='->')\n",
    "    \n",
    "    # Draw labels\n",
    "    nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')\n",
    "    \n",
    "    plt.title('Association Rules Network (Top 15 Rules)', fontsize=16)\n",
    "    plt.axis('off')\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "except ImportError:\n",
    "    print(\"  NetworkX not installed. Run: pip install networkx\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Key Insights\n",
    "\n",
    "### Observations:\n",
    "1. **Top Rule:** ...\n",
    "2. **Strongest Lift:** ...\n",
    "3. **Most Common Antecedent:** ...\n",
    "\n",
    "### Next Steps:\n",
    "- Implement sequential pattern mining (Week 2)\n",
    "- Build uplift model (Week 4)\n",
    "- Integrate into decision engine (Week 5)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}